<h1 style="text-align:center; color:#e64500ff;">Class 08 Out-of-Sample</h1>

学习了样本外预测，ENC-NEW、MSE-F检验

# 样本外预测 Out-Of-Sample

## 引言：为什么需要样本外预测？

我们前面所做的所有回归叫做**样本内回归 (In-Sample Regression)**。样本内回归使用全部可用数据来估计模型参数，然后评估模型的拟合效果。

## 样本内预测的局限性

### 1. 过度拟合问题 (Overfitting)
样本内预测容易产生过度拟合，模型可能捕捉到数据中的噪音而非真实的规律。这会导致：
* 样本内 $R^2$ 过高，给人虚假的信心
* 模型在新数据上表现很差
* 参数估计不稳定

### 2. 前瞻性偏差 (Look-Ahead Bias)
在实际投资中，我们不可能使用“未来”的数据。如果用全样本数据估计参数后再评估预测能力，就等于偷看了答案，这在学术上是不被接受的。

### 3. 真实世界的约束
在现实中，投资者只能：
* 站在当前时间点 $t$
* 使用截至 $t$ 时刻的历史数据
* 做出关于 $t+1$ 的预测决策

**因此，样本外预测 (Out-of-Sample Prediction) 才能真实反映模型的预测能力。**

In [35]:
import numpy as np # 数据处理最重要的模块
import pandas as pd # 数据处理最重要的模块
import scipy.stats as stats # 统计模块
import scipy
# import pymysql  # 导入数据库模块

from datetime import datetime # 时间模块
import statsmodels.formula.api as smf  # OLS regression

# import pyreadr # read RDS file

from matplotlib import style
import matplotlib.pyplot as plt  # 画图模块
import matplotlib.dates as mdates

from matplotlib.font_manager import FontProperties # 作图中文
from pylab import mpl
#mpl.rcParams['font.sans-serif'] = ['SimHei']
#plt.rcParams['font.family'] = 'Times New Roman'

#输出矢量图 渲染矢量图
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from IPython.core.interactiveshell import InteractiveShell # jupyter运行输出的模块

#显示每一个运行结果
InteractiveShell.ast_node_interactivity = 'all'

#设置行不限制数量
#pd.set_option('display.max_rows',None)

#设置列不限制数量
pd.set_option('display.max_columns', None)

In [36]:
# 导入市场回报率数据
from pandas.tseries.offsets import MonthEnd
Market_ret = pd.read_csv('./Marketret_mon_stock2024.csv')
Market_ret['month'] = pd.to_datetime(Market_ret['month'], format='%b %Y') + MonthEnd(0)
Market_ret.set_index('month', inplace=True)
Market_ret.sort_index(inplace=True)
Market_ret = Market_ret.drop(columns=['Unnamed: 0'])
Market_ret

,MarketR,MarketR_e,rfmonth,ret,ret_e,marketret3,marketret6,marketret12,Q
month,,,,,,,,,
1991-01-31,0.029998,0.036554,0.006930,0.023068,0.029624,-0.084127,-0.305662,0.254049,1991 Q1
1991-02-28,0.010203,0.021860,0.006930,0.003273,0.014930,-0.183573,-0.384745,0.241492,1991 Q1
1991-03-31,-0.099663,-0.060425,0.006930,-0.106593,-0.067355,-0.252928,-0.445049,0.288857,1991 Q1
1991-04-30,-0.079652,-0.031449,0.006651,-0.086303,-0.038100,-0.234776,-0.394937,0.691749,1991 Q2
1991-05-31,-0.074521,0.005375,0.006092,-0.080613,-0.000717,-0.236294,0.181673,1.542701,1991 Q2
...,...,...,...,...,...,...,...,...,...
2024-08-31,-0.031468,-0.033685,0.001118,-0.032586,-0.034803,0.151050,NaN,NaN,2024 Q3
2024-09-30,0.195992,0.227078,0.001118,0.194874,0.225960,0.200147,NaN,NaN,2024 Q3
2024-10-31,-0.003409,0.070220,0.001118,-0.004527,0.069102,0.000228,NaN,NaN,2024 Q4


In [37]:
inflation = pd.read_csv('./inflation.csv')
inflation['month'] = pd.to_datetime(inflation['month'],format='%Y/%m/%d')
inflation.set_index('month',inplace=True)
inflation.sort_values(by=['month'],axis=0,ascending=True)

,cpi
month,
1987-01-31,5.1
1987-02-28,5.4
1987-03-31,5.8
1987-04-30,6.7
1987-05-31,7.6
...,...
2025-03-31,-0.1
2025-04-30,-0.1
2025-05-31,-0.1


In [38]:
# 导入日度市场回报率数据并计算已实现波动率
Market_ret_day = pd.read_excel('./Marketret_day_stock2024.xlsx')
Market_ret_day['Day'] = pd.to_datetime(Market_ret_day['Day'],format='%Y-%m-%d')
Market_ret_day.set_index('Day',inplace=True)
Market_ret_day.sort_index(inplace=True)

# Calculate monthly realized variance (RV) by summing squared excess returns
Market_variance = Market_ret_day.resample('ME').apply(lambda df: pd.Series({
    'RV': (df['er']**2).sum(),
    'RV1': (df['er']**2).sum() + 2*(df['er']*df['er1']).sum(),
    'RV2': (df['er']**2).sum() + 2*(df['er']*df['er1']).sum() + 2*(df['er']*df['er2']).sum(),
    'RV3': (df['er']**2).sum() + 2*(df['er']*df['er1']).sum() + 2*(df['er']*df['er2']).sum() + 2*(df['er']*df['er3']).sum()
}))

Market_variance['var'] = Market_ret_day.resample('ME')['er'].var()
Market_variance.index.name = 'month'

# if RV3 <0, set to RV2 if RV2 <0, set to RV1 if RV1 <0, set to RV
Market_variance['RV1'] = Market_variance['RV1'].where(Market_variance['RV1']>=0, Market_variance['RV'])
Market_variance['RV2'] = Market_variance['RV2'].where(Market_variance['RV2']>=0, Market_variance['RV1'])
Market_variance['RV3'] = Market_variance['RV3'].where(Market_variance['RV3']>=0, Market_variance['RV2'])
Market_variance

,RV,RV1,RV2,RV3,var
month,,,,,
1990-12-31,0.011709,0.030732,0.046608,0.058424,0.000395
1991-01-31,0.000690,0.001566,0.002593,0.003461,0.000030
1991-02-28,0.000418,0.000707,0.000853,0.000763,0.000024
1991-03-31,0.001229,0.002925,0.004650,0.006369,0.000014
1991-04-30,0.000855,0.001973,0.003364,0.004531,0.000010
...,...,...,...,...,...
2024-08-31,0.000881,0.000823,0.000487,0.000487,0.000040
2024-09-30,0.012481,0.023868,0.034838,0.041347,0.000593
2024-10-31,0.011652,0.007954,0.006707,0.006707,0.000685


In [39]:
# 导入价格比率数据
price_dividend = pd.read_csv('./Price_dividend_mon2024.csv')
price_dividend['month'] = pd.date_range(start='1990-12-31', end='2024-12-31', freq='ME')
price_dividend.set_index('month', inplace=True)
price_dividend.sort_index(inplace=True)
price_dividend = price_dividend.drop(columns=['Unnamed: 0'])

price_earning = pd.read_csv('./Price_earnings_mon2024.csv')
price_earning['month'] = pd.date_range(start='1991-01-31', end='2024-12-31', freq='ME')
price_earning.set_index('month', inplace=True)
price_earning.sort_index(inplace=True)

price_bookvalue = pd.read_csv('./Price_bookvalue_mon2024.csv')
price_bookvalue['month'] = pd.date_range(start='1990-12-31', end='2024-12-31', freq='ME')
price_bookvalue.set_index('month', inplace=True)
price_bookvalue.sort_index(inplace=True)
price_bookvalue

,pb
month,
1990-12-31,1.886568
1991-01-31,2.080065
1991-02-28,1.949285
1991-03-31,1.560324
1991-04-30,1.970330
...,...
2024-08-31,0.097936
2024-09-30,0.275764
2024-10-31,0.254341


- to_v: Turnover by Volume (基于成交量的换手率)
- to_m: Turnover by Money / Amount (成交金额 / 基于金额的周转率)
### to_v 计算公式
$$\text{to\_v} = \frac{\text{成交量}}{\text{流通股本}} \times 100\%$$

### to_m 计算公式
$$\text{to\_m} = \frac{\text{成交金额}}{\text{流通市值}} \times 100\%$$

In [40]:
Turnover = pd.read_csv('./Turnover_mon2024.csv')
Turnover['month'] = pd.date_range(start='1990-12-31', periods=len(Turnover), freq='ME')
Turnover.set_index('month', inplace=True)
Turnover.sort_index(inplace=True)
Turnover

,to_v,to_m
month,,
1990-12-31,0.009403,0.009754
1991-01-31,0.012623,0.031899
1991-02-28,0.042994,0.046496
1991-03-31,0.006471,0.016755
1991-04-30,0.014430,0.026691
...,...,...
2024-08-31,0.205272,0.203924
2024-09-30,0.235540,0.231543
2024-10-31,0.424766,0.464635


In [41]:
# 合并所有数据
reg_data = pd.merge(Market_ret,inflation,on = 'month')
reg_data = pd.merge(reg_data,Market_variance,on = 'month')
reg_data = pd.merge(reg_data,price_dividend,on = 'month')
reg_data = pd.merge(reg_data,price_earning,on = 'month')
reg_data = pd.merge(reg_data,price_bookvalue,on = 'month')
reg_data = pd.merge(reg_data,Turnover,on = 'month')

reg_data = reg_data[['MarketR','rfmonth','ret','cpi','RV','RV1','RV2','RV3','var','pd','pe','pb','to_v','marketret3','marketret6','marketret12']]
reg_data

,MarketR,rfmonth,ret,cpi,RV,RV1,RV2,RV3,var,pd,pe,pb,to_v,marketret3,marketret6,marketret12
month,,,,,,,,,,,,,,,,
1991-01-31,0.029998,0.006930,0.023068,2.2,0.000690,0.001566,0.002593,0.003461,0.000030,NaN,4.466562,2.080065,0.012623,-0.084127,-0.305662,0.254049
1991-02-28,0.010203,0.006930,0.003273,1.0,0.000418,0.000707,0.000853,0.000763,0.000024,NaN,4.501042,1.949285,0.042994,-0.183573,-0.384745,0.241492
1991-03-31,-0.099663,0.006930,-0.106593,1.6,0.001229,0.002925,0.004650,0.006369,0.000014,NaN,4.112082,1.560324,0.006471,-0.252928,-0.445049,0.288857
1991-04-30,-0.079652,0.006651,-0.086303,1.3,0.000855,0.001973,0.003364,0.004531,0.000010,NaN,4.111121,1.970330,0.014430,-0.234776,-0.394937,0.691749
1991-05-31,-0.074521,0.006092,-0.080613,3.6,0.000480,0.001262,0.002119,0.002949,0.000005,NaN,4.182871,1.901101,0.039698,-0.236294,0.181673,1.542701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,-0.031468,0.001118,-0.032586,0.6,0.000881,0.000823,0.000487,0.000487,0.000040,10.728048,2.502897,0.097936,0.205272,0.151050,NaN,NaN
2024-09-30,0.195992,0.001118,0.194874,0.4,0.012481,0.023868,0.034838,0.041347,0.000593,10.889979,2.675435,0.275764,0.235540,0.200147,NaN,NaN
2024-10-31,-0.003409,0.001118,-0.004527,0.3,0.011652,0.007954,0.006707,0.006707,0.000685,10.862945,2.635010,0.254341,0.424766,0.000228,NaN,NaN


In [42]:
# 创建滞后变量
reg_data['lRV'] = reg_data['RV'].shift(1)
reg_data['lRV1'] = reg_data['RV1'].shift(1)
reg_data['lRV2'] = reg_data['RV2'].shift(1)
reg_data['lRV3'] = reg_data['RV3'].shift(1)
reg_data['lcpi'] = reg_data['cpi'].shift(2)
reg_data['lpd'] = reg_data['pd'].shift(1)
reg_data['lpe'] = reg_data['pe'].shift(1)
reg_data['lpb'] = reg_data['pb'].shift(1)
reg_data['lto_v'] = reg_data['to_v'].shift(1)
reg_data

,MarketR,rfmonth,ret,cpi,RV,RV1,RV2,RV3,var,pd,pe,pb,to_v,marketret3,marketret6,marketret12,lRV,lRV1,lRV2,lRV3,lcpi,lpd,lpe,lpb,lto_v
month,,,,,,,,,,,,,,,,,,,,,,,,,
1991-01-31,0.029998,0.006930,0.023068,2.2,0.000690,0.001566,0.002593,0.003461,0.000030,NaN,4.466562,2.080065,0.012623,-0.084127,-0.305662,0.254049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991-02-28,0.010203,0.006930,0.003273,1.0,0.000418,0.000707,0.000853,0.000763,0.000024,NaN,4.501042,1.949285,0.042994,-0.183573,-0.384745,0.241492,0.000690,0.001566,0.002593,0.003461,NaN,NaN,4.466562,2.080065,0.012623
1991-03-31,-0.099663,0.006930,-0.106593,1.6,0.001229,0.002925,0.004650,0.006369,0.000014,NaN,4.112082,1.560324,0.006471,-0.252928,-0.445049,0.288857,0.000418,0.000707,0.000853,0.000763,2.2,NaN,4.501042,1.949285,0.042994
1991-04-30,-0.079652,0.006651,-0.086303,1.3,0.000855,0.001973,0.003364,0.004531,0.000010,NaN,4.111121,1.970330,0.014430,-0.234776,-0.394937,0.691749,0.001229,0.002925,0.004650,0.006369,1.0,NaN,4.112082,1.560324,0.006471
1991-05-31,-0.074521,0.006092,-0.080613,3.6,0.000480,0.001262,0.002119,0.002949,0.000005,NaN,4.182871,1.901101,0.039698,-0.236294,0.181673,1.542701,0.000855,0.001973,0.003364,0.004531,1.6,NaN,4.111121,1.970330,0.014430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,-0.031468,0.001118,-0.032586,0.6,0.000881,0.000823,0.000487,0.000487,0.000040,10.728048,2.502897,0.097936,0.205272,0.151050,NaN,NaN,0.001878,0.001692,0.002073,0.001693,0.2,10.778043,2.544499,0.127471,0.211110
2024-09-30,0.195992,0.001118,0.194874,0.4,0.012481,0.023868,0.034838,0.041347,0.000593,10.889979,2.675435,0.275764,0.235540,0.200147,NaN,NaN,0.000881,0.000823,0.000487,0.000487,0.5,10.728048,2.502897,0.097936,0.205272
2024-10-31,-0.003409,0.001118,-0.004527,0.3,0.011652,0.007954,0.006707,0.006707,0.000685,10.862945,2.635010,0.254341,0.424766,0.000228,NaN,NaN,0.012481,0.023868,0.034838,0.041347,0.6,10.889979,2.675435,0.275764,0.235540


In [43]:
# 查看数据
reg_data.tail(10)

,MarketR,rfmonth,ret,cpi,RV,RV1,RV2,RV3,var,pd,pe,pb,to_v,marketret3,marketret6,marketret12,lRV,lRV1,lRV2,lRV3,lcpi,lpd,lpe,lpb,lto_v
month,,,,,,,,,,,,,,,,,,,,,,,,,
2024-03-31,0.016715,0.001200,0.015515,0.1,0.001174,0.001450,0.000916,0.001054,0.000058,10.831573,2.559242,0.195646,0.269898,0.022495,-0.052033,NaN,0.004431,0.005019,0.007816,0.009602,-0.8,10.813148,2.537486,0.189807,0.216073
2024-04-30,0.016724,0.001200,0.015524,0.3,0.002223,0.001160,0.002283,0.000480,0.000116,10.837655,2.605339,0.189505,0.252094,-0.034238,0.116460,NaN,0.001174,0.001450,0.000916,0.001054,0.7,10.831573,2.559242,0.195646,0.269898
2024-05-31,-0.007371,0.001200,-0.008571,0.3,0.001086,0.000785,0.000768,0.001512,0.000057,10.838736,2.599313,0.182497,0.230006,-0.049633,0.094295,NaN,0.002223,0.001160,0.002283,0.000480,0.1,10.837655,2.605339,0.189505,0.252094
2024-06-30,-0.039499,0.001200,-0.040699,0.2,0.000934,0.000445,0.000867,0.001179,0.000047,10.770461,2.557868,0.136221,0.184419,-0.072709,0.113343,NaN,0.001086,0.000785,0.000768,0.001512,0.3,10.838736,2.599313,0.182497,0.230006
2024-07-31,0.000560,0.001184,-0.000624,0.5,0.001878,0.001692,0.002073,0.001693,0.000085,10.778043,2.544499,0.127471,0.211110,0.155581,0.156368,NaN,0.000934,0.000445,0.000867,0.001179,0.3,10.770461,2.557868,0.136221,0.184419
2024-08-31,-0.031468,0.001118,-0.032586,0.6,0.000881,0.000823,0.000487,0.000487,0.000040,10.728048,2.502897,0.097936,0.205272,0.151050,NaN,NaN,0.001878,0.001692,0.002073,0.001693,0.2,10.778043,2.544499,0.127471,0.211110
2024-09-30,0.195992,0.001118,0.194874,0.4,0.012481,0.023868,0.034838,0.041347,0.000593,10.889979,2.675435,0.275764,0.235540,0.200147,NaN,NaN,0.000881,0.000823,0.000487,0.000487,0.5,10.728048,2.502897,0.097936,0.205272
2024-10-31,-0.003409,0.001118,-0.004527,0.3,0.011652,0.007954,0.006707,0.006707,0.000685,10.862945,2.635010,0.254341,0.424766,0.000228,NaN,NaN,0.012481,0.023868,0.034838,0.041347,0.6,10.889979,2.675435,0.275764,0.235540
2024-11-30,0.009724,0.001118,0.008606,0.2,0.004019,0.004118,0.005938,0.003775,0.000201,10.858076,2.643930,0.263924,0.493320,NaN,NaN,NaN,0.011652,0.007954,0.006707,0.006707,0.4,10.862945,2.635010,0.254341,0.424766


In [44]:
# 简单示例：使用已实现波动率预测市场收益率
data = reg_data['2000-01':'2024-12'].copy()
data = data.dropna()  # 删除缺失值
model_pre = 0 # 模型的预测误差平方和 (SSE)
mean_pre = 0 # 历史均值的预测误差平方和 (SST)

for i in range(int(len(data)/3), len(data) - 1):    #从数据的 $1/3$ 处开始（预留一部分数据作为初始训练集）
    # 选择数据
    data_reg = data[0:i]  #截取历史数据：data_reg = data[0:i]。每次循环，训练集都会包含从开头到当前时刻 i 的所有数据（扩张窗口，数据越积越多）。
    model =smf.ols(formula='ret ~ lRV3', data=data_reg).fit(displ=False) #训练模型：用截至 i 时刻的数据训练线性回归模型
    r_a = (model.predict(data[i:i+1][['lRV3']]) - data[i:i+1]['ret'])**2 # 模型预测值- 真实值 (data[i+1])
    r_b = (np.mean(data_reg['ret']) - data[i:i+1]['ret'])**2 #  历史平均值- 真实值 (data[i+1])
    r_a = r_a.values #为了剥离索引（Index），只保留单纯的数值，防止在后续累加计算时因为索引对不齐而报错或产生空值
    r_b = r_b.values
    model_pre = model_pre + r_a
    mean_pre = mean_pre + r_b

oos = 1 - model_pre/mean_pre  #相比于瞎猜平均值，我的模型让预测准确度提升了百分之几
print("使用已实现波动率(RV3)的样本外R2是:",oos)

使用已实现波动率(RV3)的样本外R2是: [-0.06932759]


- oos 是正数 ( > 0 )你的模型跑赢了“历史均值”。数值越大，预测能力越强。
- oos 是负数 ( > 0 )你的模型跑输了“历史均值”。

In [45]:
# 通用样本外预测函数 (简化版)
data = reg_data['2000-01':'2024-12'].copy()
data = data.dropna()  # 删除缺失值

def out_of_sample_simple(data,y,x,initial_sample_fractions):
    model_pre = 0
    mean_pre = 0

    initial_sample = int(len(data)*initial_sample_fractions)

    # 模型
    predictor_formula = "+".join(x)
    formula = f'{y} ~ {predictor_formula}'

    for i in range(initial_sample, len(data) - 1):
        # 选择数据
        data_reg = data[0:i]
        model =smf.ols(formula, data=data_reg).fit(displ=False)
        r_a = (model.predict(data[i:i+1][x]) - data[i:i+1][y])**2
        r_b = (np.mean(data_reg[y]) - data[i:i+1][y])**2
        r_a = r_a.values
        r_b = r_b.values
        model_pre = model_pre + r_a
        mean_pre = mean_pre + r_b

    oos = 1 - model_pre/mean_pre
    return oos

# 测试不同预测变量
print("单变量模型:")
print("RV3模型:", out_of_sample_simple(data,'ret',['lRV3'],0.5))   #初始样本比例为0.5
print("CPI模型:", out_of_sample_simple(data,'ret',['lcpi'],0.5))
print("PD模型:", out_of_sample_simple(data,'ret',['lpd'],0.5))
print("\n多变量模型:")
print("RV3+CPI:", out_of_sample_simple(data,'ret',['lRV3','lcpi'],0.5))
print("RV3+PD:", out_of_sample_simple(data,'ret',['lRV3','lpd'],0.5))
print("RV3+CPI+PD:", out_of_sample_simple(data,'ret',['lRV3','lcpi','lpd'],0.5))

单变量模型:
RV3模型: [-0.08398473]
CPI模型: [-0.02056036]
PD模型: [-0.00564408]

多变量模型:
RV3+CPI: [-0.15904403]
RV3+PD: [-0.08972757]
RV3+CPI+PD: [-0.18643754]


In [46]:
# 测试不同初始样本比例的影响
sample_fractions = np.linspace(0.1,0.9,9)

results = {}

for i in sample_fractions:
    results[i] = out_of_sample_simple(data,'ret',['lRV3','lcpi','lpd'],i)

# 打印数值i
print("初始样本比例对样本外R2的影响:")
for i, j in results.items():
    print(f"初始样本比例 {round(i,1)}: OOS R2 = {j[0]:.6f}")
#感觉比例0.3最好

初始样本比例对样本外R2的影响:
初始样本比例 0.1: OOS R2 = -0.033798
初始样本比例 0.2: OOS R2 = 0.003662
初始样本比例 0.3: OOS R2 = 0.008066
初始样本比例 0.4: OOS R2 = -0.171586
初始样本比例 0.5: OOS R2 = -0.186438
初始样本比例 0.6: OOS R2 = -0.230610
初始样本比例 0.7: OOS R2 = -0.107323
初始样本比例 0.8: OOS R2 = -0.158921
初始样本比例 0.9: OOS R2 = -0.183142


| 指标/检验 | 目的 | 关注点 | 核心问题 |
| :--- | :--- | :--- | :--- |
| **ENC-NEW** | **信息包容性检验** | 预测残差的**互补性** | 模型 $F$ 是否已经包含了模型 $G$ 的所有有用信息？ |
| **MSE-F (CW 检验)** | **相对准确性检验** | 均方误差 (MSE) 的**相对大小** | 模型 $G$ 的预测准确度是否显著优于模型 $F$？ |

In [47]:
# 完整的样本外检验（包含ENC-NEW和MSE-F统计量）
# benchmark model 是基准模型，为历史均值
# augment model 是比较模型，是我们要添加的预测变量

data = reg_data['2000-01':'2024-12'].copy()
data = data.dropna()  # 删除缺失值
benchmark = 0.00
augment = 0.00
ab = 0.00

for i in range(int(len(data) / 3), len(data) - 1):
    data_reg = data[0:i]
    model1 = smf.ols('ret ~ lRV3', data_reg).fit(displ=False)
    r_a = (model1.predict(data[i:i + 1][['lRV3']]) - data[i:i + 1]['ret'])**2
    r_b = (np.mean(data[0:i]['ret']) - data[i:i + 1]['ret'])**2 #均值
    r_ab = (model1.predict(data[i:i + 1][['lRV3']]) - data[i:i + 1]['ret']) * (
        np.mean(data[0:i]['ret']) - data[i:i + 1]['ret'])
    r_a = r_a.values
    r_b = r_b.values
    r_ab = r_ab.values
    augment = augment + r_a
    benchmark += r_b
    ab += r_ab

oos = 1 - augment / benchmark
ENCNEW = ((benchmark - ab) / augment) * (len(data) - int(len(data) / 3))
MSEF = (benchmark - augment) / augment * (len(data) - int(len(data) / 3))

print('使用已实现波动率(RV3)预测:')
print('样本外的R方是', oos)
print('样本外的ENC-NEW是', ENCNEW)
print('样本外的MSE-F是', MSEF)
#表现不佳，其预测效果不仅没有超过简单的历史平均值基准，而且显著更差。

使用已实现波动率(RV3)预测:
样本外的R方是 [-0.06932759]
样本外的ENC-NEW是 [-2.61425209]
样本外的MSE-F是 [-12.51274624]


In [48]:
# 使用多个预测变量: RV3 + PD + CPI
data = reg_data['2000-01':'2024-12'].copy()
data = data.dropna()  # 删除缺失值
benchmark = 0.00
augment = 0.00
ab = 0.00

for i in range(int(len(data) / 3), len(data) - 1):
    data_reg = data[0:i]
    model1 = smf.ols('ret ~ lRV3 + lpd + lcpi', data_reg).fit(displ=False)
    r_a = (model1.predict(data[i:i + 1][['lRV3','lpd','lcpi']]) - data[i:i + 1]['ret'])**2
    r_b = (np.mean(data[0:i]['ret']) - data[i:i + 1]['ret'])**2
    r_ab = (model1.predict(data[i:i + 1][['lRV3','lpd','lcpi']]) - data[i:i + 1]['ret']) * (
        np.mean(data[0:i]['ret']) - data[i:i + 1]['ret'])
    r_a = r_a.values
    r_b = r_b.values
    r_ab = r_ab.values
    augment = augment + r_a
    benchmark += r_b
    ab += r_ab

oos = 1 - augment / benchmark
ENCNEW = ((benchmark - ab) / augment) * (len(data) - int(len(data) / 3))
MSEF = (benchmark - augment) / augment * (len(data) - int(len(data) / 3))

print('使用RV3+PD+CPI预测:')
print('样本外的R方是', oos)
print('样本外的ENC-NEW是', ENCNEW)
print('样本外的MSE-F是', MSEF)
#多变量模型 RV3 + PD + CPI确实包含了独立于历史均值的预测信息ENC-NEW > 0，
# 但是当整体计算均方误差时，这些额外信息的价值不足以弥补模型带来的额外预测方差或偏差，
# 导致 MSE 整体反而比基准更高 R^2 < 0。

使用RV3+PD+CPI预测:
样本外的R方是 [-0.01992838]
样本外的ENC-NEW是 [11.1652462]
样本外的MSE-F是 [-3.77102739]


用函数构建更具有一般性的函数

In [49]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

def out_of_sample(data, response_var, predictor_vars): #数据，预测对象，预测变量
    benchmark = 0.00
    augment = 0.00
    ab = 0.00

    # 构建模型公式
    predictor_formula = ' + '.join(predictor_vars)
    formula = f'{response_var} ~ {predictor_formula}'

    for i in range(int(len(data) / 3), len(data) - 1):
        data_reg = data.iloc[0:i]
        model = smf.ols(formula, data_reg).fit(displ=False)
        prediction = model.predict(data.iloc[i:i + 1][predictor_vars])
        actual = data.iloc[i:i + 1][response_var].values[0]

        r_a = (prediction - actual) ** 2
        r_b = (np.mean(data_reg[response_var]) - actual) ** 2
        r_ab = (prediction - actual) * (np.mean(data_reg[response_var]) - actual)

        augment += r_a.values
        benchmark += r_b
        ab += r_ab.values

    oos = 1 - augment / benchmark
    ENCNEW = ((benchmark - ab) / augment) * (len(data) - int(len(data) / 3))
    MSEF = (benchmark - augment) / augment * (len(data) - int(len(data) / 3))

    return oos, ENCNEW, MSEF


oos, ENCNEW, MSEF = out_of_sample(reg_data['2000-01':'2024-12'], 'ret', ['lto_v','lpd'])
print('样本外的R方是', oos)
print('样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是', ENCNEW)
print('样本外的MSE-F是', MSEF)

样本外的R方是 [-0.02549371]
样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是 [1.23873003]
样本外的MSE-F是 [-4.97198737]


In [51]:
# 更具有一般性的函数（包含完整统计量）
def out_of_sample(data, response_var, predictor_vars, initial_sample_fraction):
    # 先删除缺失值
    data = data.dropna()
    
    benchmark = 0.00
    augment = 0.00
    ab = 0.00

    # 构建模型公式
    predictor_formula = ' + '.join(predictor_vars) # 将predictor_vars中的元素用+连接起来
    formula = f'{response_var} ~ {predictor_formula}' # 构建公式

    initial_sample_size = int(len(data) * initial_sample_fraction) # 计算初始样本的大小

    for i in range(initial_sample_size, len(data) - 1):
        data_reg = data.iloc[0:i] # 取出前i行的数据
        model = smf.ols(formula, data_reg).fit(displ=False) # 拟合模型
        prediction = model.predict(data.iloc[i:i + 1][predictor_vars]) # 预测值
        actual = data.iloc[i:i + 1][response_var].values[0] # 从data中取出第i行的response_var列的值

        r_a = (prediction - actual) ** 2 # 残差平方
        r_b = (np.mean(data_reg[response_var]) - actual) ** 2 # 均值平方
        r_ab = (prediction - actual) * (np.mean(data_reg[response_var]) - actual) # 残差乘均值

        augment += r_a.values   
        benchmark += r_b       
        ab += r_ab.values     

    oos = 1 - augment / benchmark
    ENCNEW = ((benchmark - ab) / augment) * (len(data) - initial_sample_size)
    MSEF = (benchmark - augment) / augment * (len(data) - initial_sample_size)

    return oos[0], ENCNEW[0], MSEF[0]

# 使用函数测试
oos, ENCNEW, MSEF = out_of_sample(reg_data['2000-01':'2024-12'], 'ret', ['lRV3','lpd'], initial_sample_fraction=1/3)
print('使用通用函数 - RV3+PD预测:')
print(f'样本外的R方是 {oos:.6f}')
print(f'样本外的ENC-NEW是 {ENCNEW:.6f}')
print(f'样本外的MSE-F是 {MSEF:.6f}')

使用通用函数 - RV3+PD预测:
样本外的R方是 -0.075198
样本外的ENC-NEW是 -3.038635
样本外的MSE-F是 -13.498102


In [52]:
# 设置不同的初始样本比例和预测变量组合
initial_sample_fractions = [0.3, 0.4, 0.5, 0.6, 0.7,0.8]
predictor_combinations = [
    ['lRV3'],
    ['lcpi'],
    ['lpd'],
    ['lRV3', 'lcpi'],
    ['lRV3', 'lpd'],
    ['lcpi', 'lpd'],
    ['lRV3', 'lcpi', 'lpd']
]

# 用于存储结果的字典
results = {}

print("不同预测变量组合和初始样本比例的样本外预测结果:\n")
print("="*80)

for predictors in predictor_combinations:
    predictor_name = '+'.join(predictors)
    results[predictor_name] = {}
    print(f"\n预测变量: {predictor_name}")
    print("-"*80)
    
    for fraction in initial_sample_fractions:
        # 调用 out_of_sample 函数
        oos, ENCNEW, MSEF = out_of_sample(reg_data['2000-01':'2024-12'], 'ret', predictors, initial_sample_fraction=fraction)
        
        # 将结果保存到字典
        results[predictor_name][fraction] = {
            'OOS': oos,
            'ENCNEW': ENCNEW,
            'MSEF': MSEF
        }
        
        # 打印结果
        print(f"初始样本比例: {fraction:.1f} | OOS R²: {oos:8.6f} | ENC-NEW: {ENCNEW:8.4f} | MSE-F: {MSEF:8.4f}")

不同预测变量组合和初始样本比例的样本外预测结果:


预测变量: lRV3
--------------------------------------------------------------------------------
初始样本比例: 0.3 | OOS R²: -0.020064 | ENC-NEW:   3.2138 | MSE-F:  -3.9928
初始样本比例: 0.4 | OOS R²: -0.055958 | ENC-NEW:  -2.4786 | MSE-F:  -9.2207
初始样本比例: 0.5 | OOS R²: -0.083985 | ENC-NEW:  -3.8136 | MSE-F: -11.2343
初始样本比例: 0.6 | OOS R²: -0.108207 | ENC-NEW:  -4.3323 | MSE-F: -11.3264
初始样本比例: 0.7 | OOS R²: 0.012960 | ENC-NEW:   0.7774 | MSE-F:   1.1423
初始样本比例: 0.8 | OOS R²: 0.013358 | ENC-NEW:   0.5288 | MSE-F:   0.7853

预测变量: lcpi
--------------------------------------------------------------------------------
初始样本比例: 0.3 | OOS R²: 0.027873 | ENC-NEW:   6.2479 | MSE-F:   5.8205
初始样本比例: 0.4 | OOS R²: -0.013061 | ENC-NEW:   2.4027 | MSE-F:  -2.2434
初始样本比例: 0.5 | OOS R²: -0.020560 | ENC-NEW:   0.1398 | MSE-F:  -2.9212
初始样本比例: 0.6 | OOS R²: -0.030626 | ENC-NEW:  -0.4084 | MSE-F:  -3.4471
初始样本比例: 0.7 | OOS R²: -0.084194 | ENC-NEW:  -1.2853 | MSE-F:  -6.7561
初始样本比例: 0.8 | OOS R²: